# Fall study basic stats

This notebook should always be run after the StudyMain from the fr.polytechnique.cmap.cnam.study.fall package.

## Loading data

In [ ]:
prs_path = "/shared/Observapur/staging/Flattening/single_table/ER_PRS_F"
mco_path = "/shared/Observapur/staging/Flattening/single_table/MCO_C"
ir_ben_r_path = "/shared/Observapur/staging/Flattening/single_table/IR_BEN_R"

In [ ]:
outcomes = spark.read.load("outcomes")
patients = spark.read.load("patients")

### Outcomes :

In [ ]:
print(outcomes.count())
outcomes.printSchema()
outcomes.show(truncate=False)

In [ ]:
outcomes.groupBy("value").count().show(truncate=False)

### Statistics with patients :

In [ ]:
total = outcomes.join(patients, "patientID").cache()

In [ ]:
print("Total count of patients with an eligible fracture in the data from 2010 to 2014 :")
print(outcomes.select("patientID").distinct().count())

In [ ]:
from pyspark.sql.functions import *
print("Total count of old patients with an eligible fracture in the data from 2010 to 2014 :")
print(total.filter(2010 - year("birthDate") > 65).distinct().count())

# Weird stuff

## NUM_ENQ in ben has multiple lines...

In [ ]:
ben = spark.read.load(ir_ben_r_path).withColumnRenamed("NUM_ENQ", "patientID")
print(ben.count())
print(ben.groupBy("patientID").count().filter(col("count")>1).count())

## Patients are missing from patients DataFrame

In [ ]:
problem = outcomes.select("patientID").distinct().subtract(patients.select("patientID")).cache()
problem.show()
print("Number of patientID that are in our outcomes dataframe but not in our patients dataframe :")
print(problem.count())

### Comparison with IR_BEN_R

We compare the patientIDs of the people in outcomes but not in patients with the NUM_ENQ in IR_BEN_R.

One factor that might explain the difference in our situation might be the age of the people. We don't have anyone older than 18 missing (besides some weird values).

In [ ]:
benPatients = ben.select("patientID").distinct()
pbBen = problem.intersect(benPatients)
print("Number of patient present in IR_BEN_R but not in the patient DF : ")
print(pbBen.count())
print("Computing age of the patient in IR_BEN_R missing from the patient DF : ")
pbBen.join(ben, "patientID").withColumn("age", lit(2010) - col("BEN_NAI_ANN")).groupBy("age").count().show(22)

In [ ]:
remaining = problem.subtract(benPatients).cache()
print("Number of problematic patients that doesn't appear in IR_BEN_R :")
print(remaining.count())

### Comparison with DCIR/ PRS

We have a very few number of patients in DCIR that are not in IR_BEN_R

In [ ]:
prs = spark.read.load(prs_path).withColumnRenamed("NUM_ENQ", "patientID")
remaining.intersect(prs.select("patientID").distinct()).count()

### Comparison with MCO

We have people in MCO with no existence in IR_BEN_R. This is kind of weird.

In [ ]:
mco = spark.read.load(mco_path).withColumnRenamed("NUM_ENQ", "patientID")
remaining.intersect(mco.select("patientID").distinct()).count()